In [1]:
import numpy as np
import os
from os import listdir
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:
#Load the 2nd dataset
import pandas as pd

#The data set : https://www.kaggle.com/datasets/sarabhian/affectnet-helper
#Rename the folder as faceEmotion_dataset_v3

# read the CSV files
df_train = pd.read_csv("faceEmotion_dataset_v3/train.csv")
df_test = pd.read_csv("faceEmotion_dataset_v3/val.csv")

num_classes = 8

# print the first few rows of the DataFrame
print(df_train.head())

   Unnamed: 0  class_label                                              array  \
0           0            0  [[[211 208 200]\n  [211 208 200]\n  [213 210 2...   
1           1            3  [[[199 116  47]\n  [209 121  48]\n  [221 125  ...   
2           2            2  [[[108 101 104]\n  [107 100 103]\n  [102  95  ...   
3           3            3  [[[255 255 255]\n  [255 255 255]\n  [255 255 2...   
4           4            0  [[[  4   7   0]\n  [  4   7   0]\n  [  4   6  ...   

                                            img_path  
0  ../input/affectnetsample/train_class/class001/...  
1  ../input/affectnetsample/train_class/class004/...  
2  ../input/affectnetsample/train_class/class003/...  
3  ../input/affectnetsample/train_class/class004/...  
4  ../input/affectnetsample/train_class/class001/...  


In [87]:
import ast
import numpy as np

# string representation of a list of 3D images
images_string = df_train["array"]

# remove extra whitespace and newline characters from the string
# images_string = images_string.apply(str.strip)
# images_string = images_string.replace('\n', '').replace(' ', '')
# images_string = images_string.apply(lambda elem : elem.replace('\n', '').replace('  ', ' ').replace(' ', ',').replace('[,', '[').replace(",,", ","))
images_string = images_string.apply(lambda elem : elem.replace('\n', '').replace('  ', ' ').replace('] [', '],['))
# print(test2)
print(images_string[0])
# print(images_string[0].split(','))


# # parse the string using ast.literal_eval()
# images_list = images_string.apply(ast.literal_eval)

# # convert each element of the list to a NumPy array
# images = images_list.apply(lambda x: np.array(x))

# # print the images
# print(images)


[[[211 208 200],[211 208 200],[213 210 202] ... [225 222 214],[226 224 215],[227 224 216]],[[212 209 201],[210 207 199],[217 214 207] ... [225 222 214],[225 222 214],[225 222 214]],[[212 209 201],[211 208 200],[216 213 205] ... [225 222 215],[223 220 213],[223 220 212]] ... [[137 162 166],[136 158 163],[134 154 159] ... [ 20 21 12],[ 23 24 15],[ 25 26 17]],[[131 155 161],[132 156 162],[133 154 160] ... [ 21 22 13],[ 22 23 14],[ 21 22 13]],[[128 152 158],[131 155 161],[132 156 162] ... [ 26 27 18],[ 23 24 15],[ 21 22 13]]]


In [134]:
def preprocess_dataArray(images_string):
    for image_str in (images_string):
        # images_string = images_string.apply(lambda elem : elem.replace('\n', '').replace('  ', ' ').replace('] [', '],['))
        # print(image_str.replace('\n', 'pppp'))
        image_str = image_str.replace('\n', 'pp')
        image_str_interieur = image_str[1:len(image_str) - 1]

        # image_str_interieur_interieurs = []
        # compteur_crochet = 0
        # for lettre in image_str_interieur:
        #     if(lettre == "[") :
        #         compteur_crochet +=1
        #     elif(lettre == "]") :
        #         compteur_crochet -=1
        
        # print("fin")
        print(image_str_interieur)
        print()
        image_str_interieur_splited = image_str_interieur.split('pppp ')
        print(image_str_interieur_splited)
        print()

        image_interieur_list = []
        for elem in image_str_interieur_splited :
            
            image_str_interieur_splited_interieur = elem[1:len(elem) - 1]
            print(elem)
            print(image_str_interieur_splited_interieur)
            print(image_str_interieur_splited_interieur.split("pp  "))

        #     print("fin")
            # image_interieur_list.append(elem.split('pp'))
        # print(image_interieur_list)
preprocess_dataArray(df_train["array"][0:1])

[[211 208 200]pp  [211 208 200]pp  [213 210 202]pp  ...pp  [225 222 214]pp  [226 224 215]pp  [227 224 216]]pppp [[212 209 201]pp  [210 207 199]pp  [217 214 207]pp  ...pp  [225 222 214]pp  [225 222 214]pp  [225 222 214]]pppp [[212 209 201]pp  [211 208 200]pp  [216 213 205]pp  ...pp  [225 222 215]pp  [223 220 213]pp  [223 220 212]]pppp ...pppp [[137 162 166]pp  [136 158 163]pp  [134 154 159]pp  ...pp  [ 20  21  12]pp  [ 23  24  15]pp  [ 25  26  17]]pppp [[131 155 161]pp  [132 156 162]pp  [133 154 160]pp  ...pp  [ 21  22  13]pp  [ 22  23  14]pp  [ 21  22  13]]pppp [[128 152 158]pp  [131 155 161]pp  [132 156 162]pp  ...pp  [ 26  27  18]pp  [ 23  24  15]pp  [ 21  22  13]]

['[[211 208 200]pp  [211 208 200]pp  [213 210 202]pp  ...pp  [225 222 214]pp  [226 224 215]pp  [227 224 216]]', '[[212 209 201]pp  [210 207 199]pp  [217 214 207]pp  ...pp  [225 222 214]pp  [225 222 214]pp  [225 222 214]]', '[[212 209 201]pp  [211 208 200]pp  [216 213 205]pp  ...pp  [225 222 215]pp  [223 220 213]pp  [223 2

In [4]:
def list_string_to_arrayImages_withColor(L, sizeX=48, sizeY=48):

    Images_Dataset = []
    for k in range(L.index[0], L.index[-1] + 1):
        # split the string into a list of strings
        # lst = df[" pixels"][k].split()
        lst = L[k].split()

        # convert the strings to integers using map() and a lambda function
        lst = list(map(lambda x: int(x), lst))

        Images_Dataset.append(lst)
    Images_Dataset = np.array(Images_Dataset)
    Images_Dataset = Images_Dataset.reshape(Images_Dataset.shape[0], sizeX , sizeY)

    return Images_Dataset

In [5]:
#Proprocess dataset and get X_train and X_test
X_train = list_string_to_arrayImages(df_train[" pixels"])
X_test = list_string_to_arrayImages(df_test[" pixels"])

print((X_train).shape)
print((X_test).shape)


(28709, 48, 48)
(7178, 48, 48)


In [6]:
#get y_train and y_test
Labels_train = list(df_train["emotion"])
Labels_test = list(df_test["emotion"])

In [7]:
#change the shape of the object labels
y_train = to_categorical(Labels_train, num_classes) 
y_test = to_categorical(Labels_test, num_classes) 

# define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

predictions = model.predict(X_test)

# calculate the accuracy of the predictions
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))

print('Accuracy:', accuracy)

Epoch 1/10
898/898 [==============================] - 59s 64ms/step - loss: 8.6876 - accuracy: 0.2584
Epoch 2/10
898/898 [==============================] - 34s 38ms/step - loss: 1.6639 - accuracy: 0.3553
Epoch 3/10
898/898 [==============================] - 32s 35ms/step - loss: 1.4980 - accuracy: 0.4287
Epoch 4/10
898/898 [==============================] - 32s 36ms/step - loss: 1.3552 - accuracy: 0.4852
Epoch 5/10
898/898 [==============================] - 33s 36ms/step - loss: 1.2626 - accuracy: 0.5231
Epoch 6/10
898/898 [==============================] - 31s 34ms/step - loss: 1.1939 - accuracy: 0.5527
Epoch 7/10
898/898 [==============================] - 32s 36ms/step - loss: 1.1195 - accuracy: 0.5874
Epoch 8/10
898/898 [==============================] - 29s 32ms/step - loss: 1.0601 - accuracy: 0.6088
Epoch 9/10
898/898 [==============================] - 29s 32ms/step - loss: 1.0669 - accuracy: 0.6051
Epoch 10/10
225/225 [==============================] - 3s 13ms/step
Accuracy: 0.36